# Dataset 20 Phase 1 ML

In [27]:
import pandas as pd 
import numpy as np
import time
import progressbar

# Feature extraction libraries 

import geopy
from geopy.geocoders import Nominatim

from IPy import IP
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

pd.options.display.max_columns = 500

/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Importing Dataset 20

In [173]:
dataset_20_test = pd.read_csv('./datasets_for_project/dataset_20/\
Webpages_Classification_test_data.csv/Webpages_Classification_test_data.csv',  
                              engine='python')

dataset_20_train = pd.read_csv('./datasets_for_project/dataset_20/\
Webpages_Classification_train_data.csv/Webpages_Classification_train_data.csv',  
                               engine='python',
                               error_bad_lines=False)

dataset_20 = pd.concat([dataset_20_train, dataset_20_test], axis=0).drop(columns=['Unnamed: 0'])

Skipping line 951643: unexpected end of data


In [174]:
index_array = pd.Series([i for i in range(len(dataset_20))])
dataset_20 = dataset_20.set_index(index_array)

In [175]:
print("######### FEATURES #########")
print(dataset_20.columns)
print('\n')
print("######### Dataset details #########")
print('Contains raw data')
print(f"Total number of samples = {len(dataset_20)}")
print(f"Malicious Samples = {len(dataset_20[dataset_20['label']=='bad'])}")
print(f"Benign Samples = {len(dataset_20[dataset_20['label']=='good'])}")
dataset_20.head()

######### FEATURES #########
Index(['url', 'url_len', 'ip_add', 'geo_loc', 'tld', 'who_is', 'https',
       'js_len', 'js_obf_len', 'content', 'label'],
      dtype='object')


######### Dataset details #########
Contains raw data
Total number of samples = 1313575
Malicious Samples = 29778
Benign Samples = 1283797


,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good
2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good


## Converting locations to coordinates 

In [176]:
def get_lat_long(unique_locations):
    latitudes = []
    longitudes = []
    geolocator = Nominatim(user_agent="location_conversion")
    with progressbar.ProgressBar(max_value=len(unique_locations)) as bar:
        for index, location in enumerate(unique_locations):
            location_info = geolocator.geocode(location)
            latitudes.append(location_info.latitude)
            longitudes.append(location_info.longitude)
            bar.update(index)
    return unique_locations, latitudes, longitudes

unique_locations, latitudes, longitudes = get_lat_long(dataset_20['geo_loc'].unique())

100% (233 of 233) |######################| Elapsed Time: 0:01:55 Time:  0:01:55


In [177]:
def latitude_long_to_df(unique_locations, latitudes, longitudes, dataset):
    dataset_20['latitude'] = pd.Series(None)
    dataset_20['longitude'] = pd.Series(None)
    with progressbar.ProgressBar(max_value=len(unique_locations)) as bar:
        for index, location in enumerate(unique_locations):
            dataset_locs = dataset[dataset['geo_loc'] == location]
            dataset_locs['latitude'] = latitudes[index]
            dataset_locs['longitude'] = longitudes[index]
            
            dataset['latitude'].iloc[dataset_locs.index] = dataset_locs['latitude']
            dataset['longitude'].iloc[dataset_locs.index] = dataset_locs['longitude']
        return dataset

dataset_20 = latitude_long_to_df(unique_locations, latitudes, longitudes, dataset_20)

/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100% (233 of 233) |######################| Elapsed Time: 0:00:00 ETA:  00:00:00


In [178]:
dataset_20 = dataset_20.drop(columns=['geo_loc'])
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882


## Checking for existence of IP in URLs

In [179]:
dataset_20['has_IP_in_URL'] = pd.Series([0 for i in range(len(dataset_20))])

In [180]:
def check_if_URLS_have_ips(dataset):
    with progressbar.ProgressBar(max_value=len(dataset)) as bar:
        for i in dataset.index:
            split_string = re.split('/|.com', dataset['url'].iloc[i])
            for word in split_string:
                try:
                    IP(word)
                    dataset.iloc[i]['has_IP_in_URL'] = 1
                except:
                    pass
            bar.update(i)
    return dataset

In [181]:
dataset_20_tmp = check_if_URLS_have_ips(dataset_20)

N/A% (0 of 1313575) |                    | Elapsed Time: 0:00:00 ETA:  --:--:--/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100% (1313575 of 1313575) |##############| Elapsed Time: 0:01:13 Time:  0:01:13


In [182]:
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0


In [183]:
dataset_20['has_IP_in_URL'].unique()

array([0])

**None of these datasets have IP addresses in the URLs, might be a good idea to add some at some point**

## Number of subdomains

In [184]:
dataset_20['number_subdomains'] = pd.Series([0 for i in dataset_20.index])

In [185]:
def get_number_subdomains(url_string):
    removed_http = url_string.replace('http://', '').replace('https://', '')
    sub_array = removed_http.split('/')[0].replace('.com','').split('.')
    return len(sub_array) - 1

In [186]:
dataset_20['number_subdomains'] = dataset_20['url'].progress_apply(lambda x: get_number_subdomains(x))

In [187]:
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0,1
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0,1
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0,2
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0,1


In [188]:
dataset_20['number_subdomains'].unique()

array([1, 2, 0, 3, 4, 5, 6, 7])

**Seems to be a good range of subdomains** 

## Hostname

In [189]:
def get_hostname(url):
    parsed_url = urllib.parse.urlparse(url)
    return parsed_url.netloc

dataset_20['hostname'] = dataset_20['url'].progress_apply(lambda x: get_hostname(x))

## Length of Hostname

In [190]:
def get_length_of_hostname(url):
    parsed_url = urllib.parse.urlparse(url)
    return len(parsed_url.netloc)

In [191]:
dataset_20['length_hostname'] = dataset_20['url'].progress_apply(lambda x: get_length_of_hostname(x))

In [192]:
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18


## Ratio of digits in URL

In [201]:
def get_ratio_digits_url(url):
    try: 
        return sum(c.isdigit() for c in url)/len(url)
    except:
        return 0

dataset_20['ratio_digits_url'] = dataset_20['url'].progress_apply(lambda x: get_ratio_digits_url(x))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18,0.0,0,0.5,0


## Having @ Symbol

In [206]:
def check_at_symbol(url):
    if '@' in url:
        return 1
    return 0

dataset_20['having_@_in_url'] = dataset_20['url'].progress_apply(lambda x: check_at_symbol(x))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18,0.0,0,0.0,0


In [207]:
dataset_20['having_@_in_url'].value_counts()

0    1313214
1        361
Name: having_@_in_url, dtype: int64

## Ratio of digits in Hostname

In [208]:
dataset_20['ratio_digits_hostname'] = dataset_20['hostname'].progress_apply(lambda x: get_ratio_digits_url(x))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18,0.0,0,0.0,0


## Number of underscores

In [209]:
dataset_20['number_underscores'] = dataset_20['url'].progress_apply(lambda x: x.count('_'))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18,0.0,0,0.0,0


## Charset 

## DNS Query times

In [167]:
# import dns.resolver
# import time

# def dns_query_time_seconds(url):
#     print(url)
#     dns_start = time.perf_counter()
#     answers = dns.resolver.query(url)
#     dns_end = time.perf_counter()
#     return dns_end - dns_start

# dataset_20['dns_query_times_seconds'] = dataset_20['url'].progress_apply(lambda x: dns_query_time_seconds(x))
# dataset_20['dns_query_times_seconds'].head()

## Encoding strings in Features for ML

In [219]:
def column_adjustor(dataset_column):
    unique_values = dataset_column.unique()
    return dataset_column.progress_apply(lambda x: np.where(unique_values == x)[0][0])

In [226]:
dataset_20['who_is'] = column_adjustor(dataset_20['who_is'])
dataset_20['https'] = column_adjustor(dataset_20['https'])
dataset_20['tld'] = column_adjustor(dataset_20['tld'])

In [8]:
dataset_20['label'] = dataset_20['label'].progress_apply(lambda x: 0 if x == 'good' else 1)

In [228]:
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,0,0,0,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18,0.000000,0,0.000000,0
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,0,0,0,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0,1,www.ddj.com,11,0.281250,0,0.000000,0
2,http://www.naef-usa.com/,24,24.232.54.41,0,0,0,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0,1,www.naef-usa.com,16,0.000000,0,0.000000,0
3,http://www.ff-b2b.de/,21,147.22.38.45,1,1,1,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0,2,www.ff-b2b.de,13,0.047619,0,0.076923,0
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,0,0,0,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0,1,us.imdb.com,11,0.200000,0,0.000000,0


## Saving dataset Checkpoint!!!

In [10]:
# dataset_20.to_csv('dataset_20_new_features.csv')

In [19]:
dataset_20 = pd.read_csv('dataset_20_new_features.csv').drop(columns = ['Unnamed: 0'])
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,hostname,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,0,0,0,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,members.tripod.com,18,0.000000,0,0.000000,0
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,0,0,0,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0,1,www.ddj.com,11,0.281250,0,0.000000,0
2,http://www.naef-usa.com/,24,24.232.54.41,0,0,0,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0,1,www.naef-usa.com,16,0.000000,0,0.000000,0
3,http://www.ff-b2b.de/,21,147.22.38.45,1,1,1,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0,2,www.ff-b2b.de,13,0.047619,0,0.076923,0
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,0,0,0,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0,1,us.imdb.com,11,0.200000,0,0.000000,0


## Train-test-split

In [20]:
X = dataset_20.drop(columns=['url', 'label', 'ip_add', 'content', 'hostname'])
y = dataset_20['label']

X.head()

,url_len,tld,who_is,https,js_len,js_obf_len,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,ratio_digits_hostname,number_underscores
0,40,0,0,0,58.0,0.0,23.973937,120.982018,0,1,18,0.000000,0,0.000000,0
1,32,0,0,0,52.5,0.0,39.783730,-100.445882,0,1,11,0.281250,0,0.000000,0
2,24,0,0,0,103.5,0.0,-34.996496,-64.967282,0,1,16,0.000000,0,0.000000,0
3,21,1,1,1,720.0,532.8,39.783730,-100.445882,0,2,13,0.047619,0,0.076923,0
4,35,0,0,0,46.5,0.0,39.783730,-100.445882,0,1,11,0.200000,0,0.000000,0


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## t-SNE 

In [22]:
# from sklearn.manifold import TSNE
# import matplotlib.patches as mpatches

# X_embedded = TSNE(n_components = 2, random_state = 0, verbose=True).fit_transform(X)


# f, ax = plt.subplots(figsize=(24,16))

# blue_patch = mpatches.Patch(color='#0A0AFF', label='Not Phishing')
# red_patch = mpatches.Patch(color='#AF0000', label='Phishing')

# ax.scatter(X_embedded[:,0], X_embedded[:,1], c=(y == 0), cmap='coolwarm', label='Not Phishing', linewidths=2)
# ax.scatter(X_embedded[:,0], X_embedded[:,1], c=(y == 1), cmap='coolwarm', label='Phishing', linewidths=2)
# ax.set_title('t-SNE', fontsize=30)

# ax.grid(True)

# ax.legend(handles=[blue_patch, red_patch])

In [23]:
# pd.DataFrame(X_embedded).to_csv('tSNE_initial.csv', sep=',')

## SVM

In [24]:
svm_model = svm.SVC(kernel='rbf', verbose=1)
svm_model.fit(X_train, y_train)

[LibSVM]

SVC(verbose=1)

In [25]:
from sklearn.metrics import accuracy_score

y_pred = svm_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9988234751314939

Optimistic first try with the SVM, but might need to add more malicious samples

In [26]:
confusion_matrix(y_test, y_pred)

array([[  9292,    510],
       [     0, 423678]])

## Random Forest Classifier

In [28]:
from sklearn.ensemble import RandomForestClassifier

rfc_model = RandomForestClassifier(n_estimators=100, verbose=1)
rfc_model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished


RandomForestClassifier(verbose=1)

In [29]:
y_pred = rfc_model.predict(X_test)
accuracy_score(y_test, y_pred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished


0.998841930423549

In [30]:
confusion_matrix(y_test, y_pred)

array([[  9440,    362],
       [   140, 423538]])

## KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=20)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

In [34]:
y_pred = knn_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9988073267509459

In [35]:
confusion_matrix(y_test, y_pred)

array([[  9288,    514],
       [     3, 423675]])

## Naive Bayes

In [36]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train)

In [37]:
y_pred = gnb.predict(X_test)
accuracy_score(y_test, y_pred)

0.9971947956076405

In [38]:
confusion_matrix(y_test, y_pred)

array([[  8699,   1103],
       [   113, 423565]])

## Running selenium to get URL images

In [1]:
# chromeOptions = Options()
# chromeOptions.headless = True

# def get_screenshot(url, index):
#     driver = webdriver.Chrome()
#     driver.get(url)
#     time.sleep(1)

#     driver.get_screenshot_as_file('./dataset_20_screens/' + str(index) + '.png')
#     driver.quit()
    
# for i in dataset_20.index:
#     try:
#         get_screenshot(dataset_20.iloc[i]['url'], i)
#     except:
#         print(i)